In [1]:
import numpy as np 
import pandas as pd
import requests
from tqdm import tqdm
import ast, json
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')
data_folder = ''
external_data_folder = ''
api = ''

- ingredient_rxcuis, string list of comma separated RxNORM identifiers for the ingredients
- ingredient_names, string list of comma separated ingredients in the drug product (most are single but many combination medications as well)
- num_ingredients, number of ingredients in this drug product
- pt_meddra_id, the MedDRA preferred term code for the adverse reaction term identified
- pt_meddra_term, the MedDRA preferred term for the adverse reaction term identified
- pt_meddra_jp_term, the MedDRA/J preferred term for the adverse reaction term identified 
- percent_labels, the proportion of labels for which this adverse reaction was extracted (0-1)
- num_labels, the number of active labels for this ingredient or ingredient combination

In [2]:
kegg_df = pd.read_csv(data_folder+'kegg_rx_drug_data.csv')
kegg_df.columns = ['product', 'ingredient', 'indication', 'kegg_id', 'japic_code']
condition_df = pd.read_csv(data_folder+'ade/rx_ade_table.csv')
conditions = condition_df[['japic_code', 'ade', 'category', 'percentage']]
conditions = conditions.merge(kegg_df[['japic_code', 'kegg_id']], on = 'japic_code', how = 'left')
unique_ades = pd.read_csv(data_folder+'ade/rx_raw_ade_list.csv')
unique_ades['SDUI'] = unique_ades.SDUI.apply(lambda x: ast.literal_eval(x) if str(x)!='nan' else None)
unique_ades = unique_ades[['ade', 'SDUI']].explode('SDUI')
conditions = conditions.merge(unique_ades[['ade', 'SDUI']], on = 'ade', how = 'left')
conditions = conditions[['japic_code', 'percentage', 'SDUI']]
conditions = conditions[conditions.SDUI.notna()]
conditions.japic_code = conditions.japic_code.apply(lambda x: str(int(x)))
print(conditions.shape)
conditions.head()

(3978123, 3)


,japic_code,percentage,SDUI
0,70793,1％未満,10037844.0
1,70793,頻度不明,10037087.0
2,70793,1％未満,10012735
3,70793,1％未満,10028813
4,70793,1％未満,10047700


## Map Drgs

In [45]:
drug_df = pd.read_csv(data_folder+'rx_drug_overview_parsed.csv')
drug_df['KEGG DRUG'] = drug_df['KEGG DRUG'].apply(lambda x: BeautifulSoup(x) if str(x) != 'nan' else None)
drug_df['KEGG DRUG'] = drug_df['KEGG DRUG'].apply(lambda x: [i.text for i in x.find_all('a') if i.text[0] == 'D'][0] if x != None else None)
drug_df['en_name'] = drug_df['欧文一般名'].apply(lambda x: x.lower() if str(x) != 'nan' else None)
drug_df.head()

,japic_code,version,type,総称名,一般名,欧文一般名,薬効分類名,薬効分類番号,ATCコード,KEGG DRUG,KEGG DGROUP,en_name
0,70830,2023年4月 作成（第1版）,1,オファコル,コール酸,Cholic Acid,先天性胆汁酸代謝異常症治療薬,3999,NaN,None,NaN,cholic acid
1,70827,2023年5月 改訂（第1版）,1,テルミサルタン,テルミサルタン,Telmisartan,胆汁排泄型持続性AT1受容体ブロッカー,2149,C09CA07,D00627,"<td>\n<div>\n<a href=""/entry/DG01495+-ja"">DG01...",telmisartan
2,70826,2023年5月 改訂（第1版）,1,ソリューゲン,NaN,NaN,酢酸リンゲル液,3319,NaN,None,NaN,None
3,70825,2023年5月 改訂（第1版）,1,ソリューゲン,NaN,NaN,酢酸リンゲル液,3319,NaN,None,NaN,None
4,70824,2023年4月 作成（第1版）,1,メフィーゴ,ミフェプリストン\nミソプロストール,Mifepristone\nMisoprostol,人工妊娠中絶用製剤,2499,NaN,None,NaN,mifepristone\nmisoprostol


In [49]:
drug_filtered = drug_df[['japic_code', 'version', '総称名', '薬効分類名', 'ATCコード', 'en_name']]
drug_filtered.to_csv(data_folder+'final/drug_table.csv', index=False)
drug_filtered.head(1)

,japic_code,version,総称名,薬効分類名,ATCコード,en_name
0,70830,2023年4月 作成（第1版）,オファコル,先天性胆汁酸代謝異常症治療薬,NaN,cholic acid


In [40]:
rxnorm = pd.read_csv(external_data_folder+'umls_rxnorm.csv')
rxnorm = rxnorm[['CODE', 'STR']]
rxnorm.STR = rxnorm.STR.apply(lambda x: x.lower())
rxnorm.head(1)

,CODE,STR
0,1926948,"1,2-dipalmitoylphosphatidylcholine"


In [41]:
drug_df = drug_df.merge(rxnorm, left_on = 'en_name', right_on = 'STR', how = 'left')
drug_df = drug_df[['japic_code', '一般名', 'en_name', 'KEGG DRUG', 'CODE']]
drug_df['KEGG DRUG'] = drug_df['KEGG DRUG'].apply(lambda x: [i.text for i in x.find_all('a') if i.text[0] == 'D'][0] if x != None else None)
drug_df.head(1)

,japic_code,一般名,en_name,KEGG DRUG,CODE
0,70830,コール酸,cholic acid,None,1440856.0


In [42]:
kegg_map = pd.read_csv(data_folder+'drug_atc_kegg.csv')
kegg_dict = dict(zip(kegg_map.kegg, kegg_map.rxnorm))
drug_df['CODE'] = drug_df.apply(lambda x: kegg_dict(x['KEGG DRUG']) if x.CODE == None and x['KEGG DRUG'] in kegg_dict.keys() else x.CODE, axis = 1)
kegg_map.head(1)

,lv5,atc_substance,kegg,atc,r_concept,rxnorm
0,A01AA01,Sodium fluoride,D00943,A01AA01,C0037508,9873.0


In [44]:
drug_df.notna().sum()

japic_code    13965
一般名           12544
en_name        9560
KEGG DRUG     13759
CODE           5856
dtype: int64

In [ ]:
df = pd.read_csv(data_folder+'kegg_drug_info_mapped.csv')
df.head(1)

In [25]:
drug_df = pd.read_csv(data_folder+'drug_atc_kegg.csv')
drug_df = drug_df.merge(kegg_df, left_on = 'kegg', right_on = 'kegg_id', how = 'left')
drug_df = drug_df[drug_df.japic_code.notna()]
drug_df['japic_code'] = drug_df.japic_code.apply(lambda x: str(int(x)))
drug_df.head(1)

,lv5,atc_substance,kegg,atc,r_concept,rxnorm,product,ingredient,indication,kegg_id,japic_code
0,A01AA01,Sodium fluoride,D00943,A01AA01,C0037508,9873.0,オラブリス洗口液0.2％\n \n \n(ジーシー昭和薬品),フッ化ナトリウム,う蝕予防フッ化物洗口剤,D00943,69660


## Map to Raw

In [22]:
umls_rxnorm = pd.read_csv(external_data_folder+'umls_rxnorm.csv')[['CODE', 'STR']].drop_duplicates(subset='CODE')
umls_rxnorm.columns = ['ingredients_rxcuis', 'ingredient_names']
umls_meddra = pd.read_csv(external_data_folder+'umls_meddra_en.csv')[['CODE', 'STR']].drop_duplicates(subset='CODE')
umls_meddra.columns = ['pt_meddra_id', 'pt_meddra_term']

In [45]:
raw_tally = conditions.merge(drug_df[['japic_code','rxnorm']], on = 'japic_code', how = 'left')\
.merge(umls_rxnorm, left_on = 'rxnorm', right_on = 'ingredients_rxcuis', how = 'left')\
.merge(umls_meddra, left_on = 'SDUI', right_on = 'pt_meddra_id', how = 'left')
raw_tally = raw_tally.drop(['SDUI', 'rxnorm', 'percentage'], axis = 1)
raw_tally.to_csv(data_folder+'final/onsides_raw_table_wo_freq.csv', index=False)
raw_tally.head(1)

,japic_code,ingredients_rxcuis,ingredient_names,pt_meddra_id,pt_meddra_term
0,70793,NaN,NaN,10037844,Rash


In [50]:
onsides_df = pd.DataFrame()
for ingredient in tqdm(drug_df.rxnorm.unique().tolist()):
  product_list = drug_df[drug_df.rxnorm == ingredient].japic_code.tolist() 
  num_labels = len(product_list)
  filtered_conditions = conditions[conditions.japic_code.isin([i for i in product_list])].drop_duplicates()
  individual_df = filtered_conditions.SDUI.value_counts().reset_index()
  individual_df.columns = ['pt_meddra_id', 'percent_labels']
  individual_df['ingredients_rxcuis'] = ingredient
  individual_df['num_labels'] = num_labels
  individual_df['percent_labels'] = individual_df.percent_labels.apply(lambda x: x / num_labels)
  onsides_df = onsides_df.append(individual_df)
onsides_df.head()

100%|██████████| 1316/1316 [03:23<00:00,  6.46it/s]


,pt_meddra_id,percent_labels,ingredients_rxcuis,num_labels
0,10020751.0,0.192308,9873.0,26
1,10007247.0,0.076923,9873.0,26
2,10022998.0,0.076923,9873.0,26
0,10042128.0,0.037500,5499.0,80
0,10037844.0,0.089876,2358.0,968


In [51]:
onsides_df = onsides_df.merge(umls_rxnorm, on = 'ingredients_rxcuis', how = 'left')\
.merge(umls_meddra, on = 'pt_meddra_id', how = 'left')
onsides_df = onsides_df[['ingredients_rxcuis', 'ingredient_names', 'pt_meddra_id', 'pt_meddra_term', 'percent_labels', 'num_labels']]
print(onsides_df.shape)
onsides_df.head(1)

(72386, 6)


,ingredients_rxcuis,ingredient_names,pt_meddra_id,pt_meddra_term,percent_labels,num_labels
0,9873.0,sodium fluoride,10020751.0,Hypersensitivity,0.192308,26


In [52]:
onsides_df.to_csv(data_folder+'final/onsides_compiled_table_wo_freq.csv', index=False)